In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Import Libraries

In [43]:
import pandas as pd
import re
from gensim.models import Word2Vec
from nltk import word_tokenize
import gensim
from nltk.corpus import stopwords
import nltk
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from collections import Counter
from scipy.spatial.distance import cosine
import numpy as np
from gensim.models import Word2Vec, KeyedVectors

In [45]:
import logging
#used for word2vec warnings
# Disable warning messages from being printed to the console
logging.disable(logging.WARNING)

# SKILL

## Reading Skills

In [ ]:
df_skill = pd.read_csv("ML_skills.csv", header=None, sep=";")
df_skill["skills"] = [row.split("{")[2] for row in df_skill[0]]
df_skill= df_skill.drop(columns=0,axis=1)
df_skill.head()

,skills
0,"'name': 'Anindya Saha', 'skill': ['Java', 'Pyt..."
1,"'name': 'M. Ozan Tezcan', 'skill': ['Java', 'C..."
2,"'name': 'Clare C.', 'skill': ['Software Engine..."
3,"'name': 'Shaswat Babhulgaonkar', 'skill': ['So..."
4,"'name': 'Sai Gurrapu', 'skill': ['Machine Lear..."


In [ ]:
df_skill['skills'] = df_skill['skills'].apply(lambda x: x.split(", 'skill'")[1])
def clean(text):
    cleaned_text = text[2:-3]
    a = cleaned_text.split(", ")
    a = [i[1:-1] for i in a]
    return a

df_skill["skills"] = df_skill["skills"].apply(clean)
f = lambda x: ','.join(map(str, x)) if isinstance(x, list) else x
df_skill['skills'] = df_skill['skills'].apply(f)
df_skill['skills'] = df_skill['skills'].apply(lambda x: x.replace("'", ''))
df_skill['skills'] = df_skill['skills'].str.lower()

In [ ]:
def remove_duplicate_words(string):
  words = string.split(",")
  unique_words = []

  # iterate through the list of words
  for word in words:
    # if the word is not in the list of unique words, append it
    if word not in unique_words:
      unique_words.append(word)

  # join the list of unique words into a string and return it
  return ", ".join(unique_words)

In [ ]:
df_skill["skills"] = df_skill["skills"].apply(remove_duplicate_words) #same skills in row
df_skill

,skills
0,"java, python, machine learning, data mining, r..."
1,"java, c++, matlab, digital signal processors, ..."
2,"software engineering, artificial intelligence ..."
3,"software development, machine learning, deep l..."
4,"machine learning, leadership, research and dev..."
...,...
72,"python, machine learning, sql, recommender sys..."
73,"recherche opérationnelle, c++, mathématiques, ..."
74,"python (programming language), mysql, java, sq..."
75,"python (programming language), sql, machine le..."


## Frequency of Skills

In [ ]:
df_skill['words'] = df_skill['skills'].apply(lambda x: x.split(","))
word_freq = {}

In [ ]:
for row in df_skill['words']:
    for word in row:
        if word not in word_freq:
            word_freq[word] = 1
        else:
            word_freq[word] += 1
for word, freq in word_freq.items():
    print(f'{word}: {freq}')

java: 8
 python: 48
 machine learning: 41
 data mining: 23
 redis: 1
 natural language processing: 8
 hadoop: 15
 sqoop: 1
 hive: 7
 apache spark: 12
 sql: 40
 drools: 1
 jbpm: 1
 j2ee web services: 1
 ejb: 1
 spring: 1
 oracle: 2
 eclipse: 2
 hibernate: 3
 mysql: 22
 jprofiler: 1
 jmx: 1
 web services: 4
 soa: 1
 ibm rational software architect: 1
 sdlc: 1
 multithreading: 1
 solution architecture: 1
 cloud computing: 5
 ooad: 1
 junit: 2
 maven: 2
 github: 5
 tomcat: 1
 uml: 1
 jboss application server: 1
 weblogic: 1
 pl/sql: 4
 open source: 1
 enterprise software: 1
 c++: 45
 matlab: 30
 digital signal processors: 2
 digital signal processing: 1
 xilinx: 1
 c: 34
 microprocessors: 1
 logisim: 1
 avr: 1
 microsoft office: 23
 deep learning: 32
 computer vision: 17
software engineering: 1
 artificial intelligence (ai): 14
 software development: 12
 front-end development: 1
 algorithms: 24
 object oriented design: 1
 object-oriented programming (oop): 2
 web applications: 3
 data scie

In [ ]:
sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
top_15_words = sorted_word_freq[:15]
for word, freq in top_15_words:
    print(f'{word}: {freq}')

#java ent edition, java web services,core java,java 8

 java: 50
 python: 48
 c++: 45
 machine learning: 41
 sql: 40
 c: 34
 deep learning: 32
 matlab: 30
 algorithms: 24
 r: 24
 data analysis: 24
 data mining: 23
 microsoft office: 23
 tensorflow: 23
 mysql: 22


In [ ]:
df_skill= df_skill.drop(columns="words",axis=1)


## Preprocess

In [ ]:
lemmatizer = WordNetLemmatizer()
def tokenize_skills(string):
    tokens = re.split(r'[,\s]+', string)
    tokens = [token for token in tokens if token]# Remove any empty tokens
    return tokens
# Remove stop words
def remove_stop_words_skills(tokens):
    clean_tokens = [token for token in tokens if token.lower() not in stopwords.words('english')]
    return clean_tokens

def lemmatize_word(word):
  return lemmatizer.lemmatize(word)
def lemmatize_list(words):
  lemmas = []
  for word in words:
    lemma = lemmatize_word(word)
    lemmas.append(lemma)
  return lemmas

In [ ]:
df_skill.head()

,skills
0,"java, python, machine learning, data mining, r..."
1,"java, c++, matlab, digital signal processors, ..."
2,"software engineering, artificial intelligence ..."
3,"software development, machine learning, deep l..."
4,"machine learning, leadership, research and dev..."


In [ ]:
df_skill['skills'] = df_skill['skills'].apply(tokenize_skills)
df_skill['skills'] = df_skill['skills'].apply(lemmatize_list)
df_skill['skills'] = df_skill['skills'].apply(remove_stop_words_skills)

In [ ]:
df_skill.head()

,skills
0,"[java, python, machine, learning, data, mining..."
1,"[java, c++, matlab, digital, signal, processor..."
2,"[software, engineering, artificial, intelligen..."
3,"[software, development, machine, learning, dee..."
4,"[machine, learning, leadership, research, deve..."


## Skill model

In [ ]:
corpus_skill = df_skill['skills'] .tolist()

In [ ]:
len(corpus_skill)

77

In [ ]:
corpus_skill

In [ ]:
model_skill = Word2Vec(corpus_skill, size=10, window=1, min_count=1, workers=4)


In [ ]:
model_skill.train(corpus_skill, total_examples=len(corpus_skill), epochs=5)

(8917, 14225)

In [ ]:
similar_words = model_skill.wv.most_similar('machine')
similar_words

[('database', 0.9554028511047363),
 ('hive', 0.9436065554618835),
 ('processing', 0.9122926592826843),
 ('language', 0.887672483921051),
 ('scikit-learn', 0.8817493319511414),
 ('server', 0.8770807385444641),
 ('matlab', 0.8650325536727905),
 ('science', 0.8649402856826782),
 ('amazon', 0.8614377975463867),
 ('linux', 0.8611794710159302)]

In [ ]:
"""vectors3 = np.array(model3.wv.vectors)
vectors4 = np.array(model4.wv.vectors)
vectors5 = np.array(model5.wv.vectors)

# Get the length of the longest vector
max_length = max(len(vectors3), len(vectors5))

# Pad the shorter vectors with zeros
padded_vectors3 = np.pad(vectors3, ((0, max_length - len(vectors3)), (0, 0)), 'constant')
padded_vectors4 = np.pad(vectors5, ((0, max_length - len(vectors5)), (0, 0)), 'constant')

print(max_length)
print(len(padded_vectors3))
print(len(padded_vectors4))


# Flatten the padded vectors into 1D arrays
flattened_vectors3 = padded_vectors3.flatten()
flattened_vectors4 = padded_vectors4.flatten()
"""

## Cosine

In [ ]:
# Calculate the similarity
similarity = 1 - cosine(flattened_vectors3, flattened_vectors4)
print(similarity)

0.004382339306175709


# COURSE OUTCOMES

## Reading Outcomes

In [105]:
df_udemy = pd.read_csv("udemy_ML.csv", header=None, sep=";")
df_udemy["courses"] = [row.split("{")[2] for row in df_udemy[0]]
df_udemy= df_udemy.drop(columns=0,axis=1)
df_udemy["courses"]=df_udemy["courses"].str.replace("'Outcomes':","")
df_udemy["courses"]=df_udemy["courses"].str.replace("[","")
df_udemy["courses"]=df_udemy["courses"].str.replace("]","")
df_udemy["courses"]=df_udemy["courses"].str.replace("}","")
df_udemy["courses"]=df_udemy["courses"].str.replace("\"","'")
df_udemy["courses"]=df_udemy["courses"].apply(lambda x : x + ',')

#df_udemy['courses'] = df_udemy['courses'].apply(lambda x: x.split("'Outcomes': ")[1])

df_udemy.head()

,courses
0,"'Master Machine Learning on Python & R', 'Hav..."
1,'Use Python for Data Science and Machine Lear...
2,'You will learn how to use data science and m...
3,'The course provides the entire toolbox you n...
4,'Learn multiple techniques for missing data i...


In [106]:
first_df = pd.DataFrame(columns=['courses', 'course'])
pattern = "'(.*?)',"
for i, row in enumerate(df_udemy["courses"]):
    lst = re.findall(pattern, row)
    temp_df = pd.DataFrame({'courses': lst, 'course': [f'course {i}']*len(lst)})
    first_df = first_df.append(temp_df, ignore_index=True)

first_df= first_df.drop(columns="course",axis=1)
df = first_df.copy(deep=True)

## Preprocess 

In [107]:
df['courses'] = df['courses'].str.lower()


In [108]:
stop_words = stopwords.words('english')
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens
# Remove stop words
def remove_stop_words(tokens):
    clean_tokens = [token for token in tokens if token.lower() not in stopwords.words('english')]
    return clean_tokens
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words
def lemmatize_words(words):
  lemmatizer = WordNetLemmatizer()
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  return lemmatized_words

"""def stem_words(words):
  stemmer = PorterStemmer()
  stemmed_words = [stemmer.stem(word) for word in words]
  return stemmed_words"""

'def stem_words(words):\n  stemmer = PorterStemmer()\n  stemmed_words = [stemmer.stem(word) for word in words]\n  return stemmed_words'

In [109]:
df['courses'] = df['courses'].apply(tokenize)

In [110]:
df.head()

,courses
0,"[master, machine, learning, on, python, &, r]"
1,"[have, a, great, intuition, of, many, machine,..."
2,"[make, accurate, predictions]"
3,"[make, powerful, analysis]"
4,"[make, robust, machine, learning, models]"


In [111]:
df['courses'] = df['courses'].apply(remove_stop_words)
df['courses'] = df['courses'].apply(remove_punctuation)
df['courses'] = df['courses'].apply(lemmatize_words)


In [112]:
df.head()

,courses
0,"[master, machine, learning, python, r]"
1,"[great, intuition, many, machine, learning, mo..."
2,"[make, accurate, prediction]"
3,"[make, powerful, analysis]"
4,"[make, robust, machine, learning, model]"


## Model

In [113]:
corpus = df['courses'] .tolist()

In [ ]:
corpus

### Finding optimal hyperparameters

size: The dimensionality of the word vectors that the model will learn

 window: The size of the context window used to predict the target word

 min_count: The minimum number of times a word must appear in the corpus to be included in the model

 workers: The number of worker threads to use for training the mode

In [115]:
def train_word2vec(corpus, word1, word2, min_size=50, max_size=100, min_window=5, max_window=10, min_count=1, workers=4):
  best_size = min_size
  best_window = min_window
  best_min_count = min_count
  best_model = None
  best_score = -float('inf')

  for size in range(min_size, max_size + 1):
    for window in range(min_window, max_window + 1):
      for min_count in range(1, 3):
        model = Word2Vec(corpus, size=size, window=window, min_count=min_count, workers=workers)
        model.train(corpus, total_examples=len(corpus), epochs=15)
        score = model.wv.similarity(word1, word2)
        if score > best_score:
          best_size = size
          best_window = window
          best_min_count = min_count
          best_model = model
          best_score = score

  print(f'Best size: {best_size}')
  print(f'Best window: {best_window}')
  print(f'Best min_count: {best_min_count}')
  return best_model


Printing top 5 frequent pair of words in corpus for using them finding the optimal hyperparameters.

In [114]:
def most_frequent_pair(corpus):
  # Flatten the corpus list of lists into a single list
  flat_corpus = [word for sublist in corpus for word in sublist]
  # Initialize a counter to count the frequency of each pair of words
  pair_counts = Counter()
  # Iterate through the flat corpus and count the frequency of each pair of words
  for i in range(len(flat_corpus)-1):
    pair = (flat_corpus[i], flat_corpus[i+1])
    pair_counts[pair] += 1
  # Find the top 5 most frequent pairs of words
  top_pairs = pair_counts.most_common(5)
  return top_pairs

print(most_frequent_pair(corpus))

[(('machine', 'learning'), 37), (('neural', 'network'), 16), (('data', 'science'), 11), (('learning', 'algorithm'), 9), (('deep', 'learning'), 8)]


We will use only three of them.
[(('machine', 'learning'), 37), (('neural', 'network'), 16), (('data', 'science'), 11), (('learning', 'algorithm'), 9), (('deep', 'learning'), 8)]


In [116]:
train_word2vec(corpus,'neural','network' ,min_size=50, max_size=100, min_window=1, max_window=10, min_count=1, workers=4)

Best size: 99
Best window: 10
Best min_count: 1


In [117]:
train_word2vec(corpus,'data','science' ,min_size=50, max_size=100, min_window=1, max_window=10, min_count=1, workers=4)

Best size: 55
Best window: 7
Best min_count: 2


In [118]:
train_word2vec(corpus,'machine','learning' ,min_size=50, max_size=100, min_window=1, max_window=10, min_count=1, workers=4)

Best size: 50
Best window: 8
Best min_count: 2


**Results for "neural network" min distance**

Best  size: 99

Best window: 10

Best min_count: 1

**Results for "data science" min distance**

Best size: 55

Best window: 7

Best min_count: 2

**Results for "machine learning" min distance**

Best size: 50

Best window: 8

Best min_count: 2



After compared 3 different similarity we decided to use neural network hyperparameters.

**With neural network hyperparameters**

- Sim(data,scince) 0.82586724

- Sim(neural,network) 0.8188135

- Sim(machine,learning) 0.9418734


**With data science hyperparameters**

- Sim(data,scince) 0.8763663

- Sim(neural,network) 0.7670939

- Sim(machine,learning) 0.946057


**With machine learning hyperparameters**

- Sim(data,scince) 0.7845713

- Sim(neural,network) 0.7333079

- Sim(machine,learning) 0.95651054




In [119]:
model = gensim.models.Word2Vec(corpus, size=99, window=10, min_count=1, workers=4)

In [120]:
model.train(corpus, total_examples=len(corpus), epochs=15)

(15029, 20445)

In [103]:
model.save("course.model")

In [121]:
similarity = model.similarity('data', 'science')
similarity

0.82586724

In [122]:
similarity = model.similarity('neural', 'network')
similarity

0.8188135

In [123]:
similarity = model.similarity('machine', 'learning')
similarity

0.9418734

In [124]:
similar_words = model.wv.most_similar('machine')
similar_words

[('learning', 0.94187331199646),
 ('data', 0.918445348739624),
 ('using', 0.9167494177818298),
 ('service', 0.9072754979133606),
 ('sagemaker', 0.9031240940093994),
 ('algorithm', 0.8981636762619019),
 ('learn', 0.8977909684181213),
 ('model', 0.8946993947029114),
 ('lambda', 0.8853323459625244),
 ('deep', 0.8831156492233276)]

In [ ]:
print(len(model.wv["machine"]))
print(model.wv["machine"].shape)
print(model.wv["machine"].ndim)

model.wv["machine"]

100
(100,)
1


array([-3.5548713e-03,  3.6473700e-03, -3.4205206e-03,  3.4715533e-03,
        2.4745730e-03,  4.1808756e-03,  3.4915418e-03, -1.9654185e-04,
        3.0310501e-03, -1.8700834e-03,  4.5849737e-03,  3.1068365e-03,
       -4.8355581e-03,  3.6351764e-04,  4.6417047e-03, -1.6600027e-03,
        4.2728903e-03, -4.2808917e-03,  3.9515682e-03, -2.8317496e-03,
       -2.6362631e-03,  3.0372853e-03,  8.3668745e-04,  3.7042142e-03,
        4.1410159e-03,  1.2338447e-03, -9.6488273e-04, -9.5823267e-04,
       -1.8911565e-03,  3.9345063e-03, -1.7520247e-03, -2.8719655e-03,
       -8.9067194e-05, -1.2254577e-04,  1.5388351e-03,  3.5948397e-03,
       -4.8795197e-04,  3.1682153e-03, -1.3300584e-03,  3.4716185e-03,
        1.7750742e-03,  4.8758835e-03, -3.2572483e-03, -3.8179050e-03,
       -4.8411097e-03,  1.2488248e-03, -2.4996279e-03,  2.2546984e-03,
       -3.7018296e-03, -1.8039325e-03, -4.1726453e-04,  3.9929617e-03,
        3.9295107e-03, -4.2859213e-03, -1.5074640e-03, -1.1725652e-03,
      

In [ ]:
print(len(model.wv["learning"]))
print(model.wv["learning"].shape)
print(model.wv["learning"].ndim)

model.wv["learning"]

100
(100,)
1


array([-3.5757835e-03, -1.9550642e-03,  4.2725555e-04,  4.0496876e-03,
       -9.3915308e-04, -3.6598570e-03,  3.4887780e-04,  4.7978014e-04,
        3.8146204e-03, -3.4307104e-03, -3.2335392e-03, -4.2028781e-03,
        4.1947741e-04, -2.7545970e-03,  4.8324959e-03,  2.7034085e-03,
       -4.2450047e-04, -9.4674004e-04,  1.8853318e-03,  3.8270871e-03,
       -3.8184747e-03, -2.3709363e-03,  5.2769732e-04,  3.1546287e-03,
        1.0086988e-03,  2.4366754e-03, -2.8738165e-03, -1.6286762e-03,
        4.6827845e-04,  9.4134906e-05,  8.7786957e-06,  4.2263521e-03,
        2.6477831e-03,  3.2830706e-03,  4.0514343e-03, -3.0289168e-04,
        1.2665197e-03, -2.2082457e-03,  1.2217036e-03,  1.8201055e-03,
        2.4717313e-03, -3.5751974e-03,  1.9922203e-03, -2.7593004e-04,
       -2.6535941e-03,  1.2317039e-03,  1.3978541e-03, -3.4642483e-03,
        2.7899621e-03, -1.1192106e-03, -3.9320234e-03,  4.3420875e-03,
        1.1351712e-03,  4.6668644e-03,  4.6275629e-05,  4.7382214e-03,
      